In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
from fake_useragent import UserAgent
from tqdm.notebook import tqdm
import time

def configure_webdriver():
    # Configurações do WebDriver com agente de usuário aleatório
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    
    # Gera um agente de usuário aleatório usando o fake-useragent
    user_agent = UserAgent().random
    chrome_options.add_argument(f'user-agent={user_agent}')
    
    # Inicializa o WebDriver
    return webdriver.Chrome(options=chrome_options)

def fetch_news_for_date(web, pesquisa, data):
    url = f'https://www.google.com/search?q={pesquisa}&sca_esv=8a38f1b162d79def&rlz=1C1CHBD_pt-PTBR1080BR1080&tbs=cdr:1,cd_min:{data},cd_max:{data},lr:lang_1pt&tbm=nws&source=lnt&lr=lang_pt&sa=X&ved=2ahUKEwjy8NbRxaqIAxVJpZUCHVnpHTgQpwV6BAgCEAc&biw=1536&bih=738&dpr=1.25'
    
    web.get(url)
    
    # Espera o carregamento completo da página
    WebDriverWait(web, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.n0jPhd.ynAwRc.MBeuO.nDgy9d'))
    )
    
    soup = BeautifulSoup(web.page_source, 'html.parser')
    
    # Ajuste aqui dependendo do HTML atual
    news = soup.find_all('div', class_='n0jPhd ynAwRc MBeuO nDgy9d')
    
    return [{'Data': data, 'Notícia': new.text} for new in news]

def get_news(pesquisa, dia=None, quant_dias=5, dias=None):
    if dia is None:
        dia = dt.datetime.now()
    pesquisa = pesquisa.replace(' ', '+')
    
    web = configure_webdriver()
    dados = []
    datas_ruins = []

    try:
        # Gera a lista de datas se não for fornecida
        if dias is None:
            dias = [(dia - dt.timedelta(days=i)).strftime('%m/%d/%Y') for i in range(1, quant_dias + 1)]
        
        with tqdm(total=len(dias), desc="Progresso", leave=True) as pbar:
            for passado in dias:
                try:
                    noticias = fetch_news_for_date(web, pesquisa, passado)
                    dados.extend(noticias)
                except Exception as e:
                    datas_ruins.append(passado)
                    print(f'Erro ao processar a data {passado}: {e}')
                
                pbar.update(1)
                
                # Adiciona um delay aleatório para evitar ser bloqueado pelo Google
                time.sleep(2)
    finally:
        web.quit()
    
    # Converte a lista para DataFrame
    return pd.DataFrame(dados), datas_ruins

In [2]:
lista = ['b3']
dfs = []
for pesquisa in lista:
    df, datas = get_news(pesquisa, quant_dias=6)
    df['Data'] = pd.to_datetime(df['Data'])
    dfs.append(df)
    print(f'Dados salvos em ../data/news/noticias_{pesquisa}.csv')

Progresso:   0%|          | 0/6 [00:00<?, ?it/s]

Dados salvos em ../data/news/noticias_bolsa de valores.csv


Progresso:   0%|          | 0/6 [00:00<?, ?it/s]

Dados salvos em ../data/news/noticias_ibovespa.csv


Progresso:   0%|          | 0/6 [00:00<?, ?it/s]

Dados salvos em ../data/news/noticias_ações.csv


Progresso:   0%|          | 0/6 [00:00<?, ?it/s]

Dados salvos em ../data/news/noticias_mercado financeiro.csv


In [3]:
import pandas as pd

news = pd.read_csv('../data/news/noticias_sentimentos.csv')

In [8]:
dfzao = pd.concat(dfs)
dfzao.drop_duplicates(inplace=True)

In [10]:
dfzao.to_csv('../data/news/noticias_atuais.csv', index=False)